Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

In [1]:
# Please add your name and University of Michagan uniqname here...

NAME = 'Aseem Sachdeva'
UMICH_UNIQNAME = 'aseemsa'

---

In [2]:
version = "v2.3.060120"

# SIADS 515 Week 3 Homework (HW3)


Think of this assignment in the following way:  it's your first day on the job and you've been given a notebook that was authored by someone who is no longer with your company.  You've been asked to fix it.  There are errors in it, and some of it was not completed by the original author.  You're lucky, though, as there are assertions
sprinkled throughout the notebook to help guide you along the way.


Top-level goal of notebook:
Read a CSV file into a pandas DataFrame and add specific columns to it.
These columns are added by applying functions to specific columns.
The columns to add include:
1. A datetime column that converts "Garmin time" to standard (unix epoch) time.  Note that Garmin doesn't use standard epoch offsets for their timestamps.  Rather than using the number of seconds that have elapsed since midnight on January 1, 1970, they use the number of seconds from midnight on December 31, 1989.

2. A conversion of "semicircles" of latitude and longitude to two different formats: degrees, minutes, seconds 3-tuples and fractional degrees.  For example, a latitude of 504719750 semicircles corresponds to a 3-tuple of degrees, minutes and seconds of (42, 18, 18.43) and 42.305121 degrees.

3. A "normalized speed" column that consists of the values for speed modified to remove outliers by replacing them with upper and lower bounds as well as normalized to z-values (i.e. by subtracting the mean from each value and dividing the result by the standard deviation).

In addition, you will need to complete a function that looks at the difference between sequential rows to determine whether the cyclist is slowing down or not.

Your task for this assignment is to debug this notebook to produce the desired results as shown in the assertions below.



In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
ride = pd.read_csv('assets/ride_final2.csv')

In [6]:
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed
0,896018545,504719750,-998493490,10.87,285.8,1.773
1,896018560,504717676,-998501870,71.85,285.0,5.533
2,896018566,504716354,-998506792,108.02,284.0,6.485
3,896018575,504714055,-998515244,170.23,284.0,6.951
4,896018584,504711900,-998523278,229.27,285.0,6.224


In [7]:
def garmin_time_to_datetime(series):
    """Convert Garmin FIT time by adding the number of 
    seconds from January 1, 1970 to December 31, 1989.
    """
  
    
    return pd.to_datetime(series + 631065600.0, unit='s')

In [8]:
# import datetime
# future_date = datetime.datetime(1989, 12, 31)
# past_date = datetime.datetime(1970,1, 1)
# difference = (future_date - past_date)
# total_seconds = difference.total_seconds()
# print(total_seconds)

In [9]:
series = ride['Timestamp']
ride['Timestamp_datetime'] = ride.Timestamp.map(garmin_time_to_datetime)


In [10]:
from numpy import datetime64
assert ride.Timestamp_datetime.values[0] == datetime64('2018-05-23T14:02:25.000000000')

In [11]:
def semicircles_to_degrees(semicircles):
    '''
    Convert semicircles to degrees
    '''
    max_32_bit_int = 2**31
    return semicircles * (180/max_32_bit_int)

In [12]:
 #semicircles_to_degrees(504719750)

In [13]:
def degrees_to_dms(degrees_fraction):
    ''' Convert degrees to degree, minute, second 3-tuples'''
    degrees = int(degrees_fraction)
    minutes = int((degrees_fraction - degrees) * 60)
    seconds = np.round((degrees_fraction - degrees - minutes/60) * 3600,2)
    return (degrees, abs(minutes), abs(seconds))

In [14]:
#degrees_to_dms(42.30512073263526)

In [15]:
def dms_to_degrees(d,m,s):
    ''' Convert degrees, minutes, seconds to fractional degrees'''
    return d+m/60+s/3600

In [16]:
assert dms_to_degrees(*degrees_to_dms(42.2833333333)) == 42.28333333333333

In [17]:
ride['Latitude_degrees'] = ride['Latitude'].map(semicircles_to_degrees)

In [18]:
ride['Longitude_degrees'] = ride['Longitude'].map(semicircles_to_degrees)

In [19]:
ride['Latitude_dms'] = ride['Latitude_degrees'].map(degrees_to_dms)

In [20]:
ride['Longitude_dms'] = ride['Longitude_degrees'].map(degrees_to_dms)

In [21]:
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25,42.305121,-83.692758,"(42, 18, 18.43)","(-83, 41, 33.93)"
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40,42.304947,-83.693460,"(42, 18, 17.81)","(-83, 41, 36.46)"
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46,42.304836,-83.693872,"(42, 18, 17.41)","(-83, 41, 37.94)"
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55,42.304643,-83.694581,"(42, 18, 16.72)","(-83, 41, 40.49)"
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04,42.304463,-83.695254,"(42, 18, 16.07)","(-83, 41, 42.92)"


In [22]:
ride.tail()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms
209,896019779,504424374,-999047484,5930.37,278.2,4.740,2018-05-23 14:22:59,42.280363,-83.739193,"(42, 16, 49.31)","(-83, 44, 21.09)"
210,896019785,504424487,-999051230,5955.96,278.0,4.059,2018-05-23 14:23:05,42.280372,-83.739507,"(42, 16, 49.34)","(-83, 44, 22.22)"
211,896019788,504425416,-999051046,5965.33,278.0,3.042,2018-05-23 14:23:08,42.280450,-83.739491,"(42, 16, 49.62)","(-83, 44, 22.17)"
212,896019791,504425984,-999050165,5973.35,278.0,2.267,2018-05-23 14:23:11,42.280498,-83.739417,"(42, 16, 49.79)","(-83, 44, 21.9)"
213,896019805,504426837,-999050457,5981.77,278.2,0.000,2018-05-23 14:23:25,42.280569,-83.739442,"(42, 16, 50.05)","(-83, 44, 21.99)"


In [23]:
from numpy import round
last_row = ride.iloc[213]
assert round(last_row.Latitude_degrees,6) == 42.280569
assert round(last_row.Longitude_degrees,6) == -83.739442
assert last_row.Latitude_dms == (42, 16, 50.05)
assert last_row.Longitude_dms == (-83,44,21.99)


In [24]:
def normalize(df, pd_series_name, nsd=2):
    '''
    Take all values that are outside some bound (mean +- 2 sd by default)
    and convert them to the appropriate bound.
    Scale the results to z-scores before returning them
    '''
    df = df.copy()
    pd_series = df[pd_series_name].astype(float)

    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)

    # Calculate the bounds
    lower_bound = avg - nsd*sd
    upper_bound = avg + nsd*sd

    # Collapse in the outliers: replace them with appropriate bound
    df.loc[pd_series < lower_bound , pd_series_name ] = lower_bound
    df.loc[pd_series > upper_bound , pd_series_name ] = upper_bound
    
    return (df[pd_series_name] - avg) / sd

In [25]:
ride['Speed_normalized'] = normalize(ride,'Speed')

In [26]:
ride.tail()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms,Speed_normalized
209,896019779,504424374,-999047484,5930.37,278.2,4.740,2018-05-23 14:22:59,42.280363,-83.739193,"(42, 16, 49.31)","(-83, 44, 21.09)",-0.464215
210,896019785,504424487,-999051230,5955.96,278.0,4.059,2018-05-23 14:23:05,42.280372,-83.739507,"(42, 16, 49.34)","(-83, 44, 22.22)",-0.764769
211,896019788,504425416,-999051046,5965.33,278.0,3.042,2018-05-23 14:23:08,42.280450,-83.739491,"(42, 16, 49.62)","(-83, 44, 22.17)",-1.213615
212,896019791,504425984,-999050165,5973.35,278.0,2.267,2018-05-23 14:23:11,42.280498,-83.739417,"(42, 16, 49.79)","(-83, 44, 21.9)",-1.555656
213,896019805,504426837,-999050457,5981.77,278.2,0.000,2018-05-23 14:23:25,42.280569,-83.739442,"(42, 16, 50.05)","(-83, 44, 21.99)",-2.000000


In [27]:
from numpy import round
assert round(ride.iloc[0].Speed_normalized,4) == -1.7737
assert ride.iloc[213].Speed_normalized == -2.0

In [28]:
def proportion_slowing(df,series_name):
    ''' Calculate the proportion of rows that represent a slower speed than the previous row'''
    length = len(df)
    df['Compare_column'] = df[series_name].shift(1)
    df['flag'] = np.where(df['Speed_normalized'] < df['Compare_column'], 'yes', 'no')
    count_of_yes = (df.flag == 'yes').sum()
    count_of_yse = count_of_yes 
    proportion = count_of_yes/length
    return proportion
    
    
    #return 0 # whoops -- ran out of time to do this before I got fired!

In [29]:
#round(proportion_slowing(ride, 'Speed_normalized'),6)


In [30]:
from numpy import round
assert round(proportion_slowing(ride,'Speed_normalized'),6) == 0.514019
assert round(proportion_slowing(ride[:10],'Speed_normalized'),6) == 0.4
assert round(proportion_slowing(ride[10:],'Speed_normalized'),6) == 0.519608

In [31]:
# This cell is here to assign points for your submission. The content will be manually graded.
assert True == True, "Assignment 3 has been submitted."